In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install sentence-transformers

In [2]:
import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [3]:
model = SentenceTransformer('snunlp/KR-Sbert-V40K-klueNLI-augSTS')
embedding_data = torch.load('/content/drive/MyDrive/gdsc/team-study/embedding_data.pt')
df = pd.read_excel('/content/drive/MyDrive/gdsc/team-study/Qusetion.xlsx')

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [9]:
sentence = "ai/ml에는 누가 있어?"
print("질문 문장 :", sentence)

sentence = sentence.replace(" ", "")
print("공백 제거 문장 :", sentence)

질문 문장 : ai/ml에는 누가 있어?
공백 제거 문장 : ai/ml에는누가있어?


In [10]:
sentence_encode = model.encode(sentence)
sentence_tensor = torch.tensor(sentence_encode)

In [11]:
cos_sim = util.cos_sim(sentence_tensor, embedding_data)
print(f"가장 높은 코사인 유사도 idx : {int(np.argmax(cos_sim))}")

가장 높은 코사인 유사도 idx : 295


In [12]:
df

,의도 (Intent),질문 (Query),답변 (Answer),비고
0,연락처,강예인에게 어떻게 연락하면 될까?,"전화번호 : 010-2397-9892, 이메일 : kangyein9892@gmail...",개인
1,연락처,강예인에게 연락하고 싶어,"전화번호 : 010-2397-9892, 이메일 : kangyein9892@gmail...",개인
2,연락처,강예인의 번호를 알려줘,"전화번호 : 010-2397-9892, 이메일 : kangyein9892@gmail...",개인
3,전공,강예인의 전공을 알고 싶어,제 1전공 : 정보보호,개인
4,전공,강예인의 전공이 뭐야?,제 1전공 : 정보보호,개인
...,...,...,...,...
321,학과,수학과 복수전공 출신은 누가 있을까?,김수민님이 수학과를 복수전공했어요 :),학과
322,학과,수학과를 복수전공한 사람에는 누가 있어?,김수민님이 수학과를 복수전공했어요 :),학과
323,학과,데이터사이언스학과를 복수전공한 사람은 누가 있어?,이서현님이 데이터사이언스학과를 복수전공했어요 :),학과
324,학과,데이터사이언스학과 복수전공 출신은 누가 있을까?,이서현님이 데이터사이언스학과를 복수전공했어요 :),학과


In [13]:
best_sim_idx = int(np.argmax(cos_sim))
selected_qes = df['질문 (Query)'][best_sim_idx]
print(f"선택된 질문 = {selected_qes}")

선택된 질문 = ai/ml부서에 누가 있어?


In [14]:
selected_qes_encode = model.encode(selected_qes)

In [15]:
score = np.dot(sentence_tensor, selected_qes_encode) / (np.linalg.norm(sentence_tensor) * np.linalg.norm(selected_qes_encode))
print(f"선택된 질문과의 유사도 = {score}")

선택된 질문과의 유사도 = 0.7871719002723694


In [16]:
answer = df['답변 (Answer)'][best_sim_idx]
print(f"\n답변 :{answer}")


답변 :ai/ml 부서에는 다음과 같은 사람들이 있습니다 (배지윤, 이재연, 정다혜, 김수민, 오윤선, 이서현)
